# HIGH-DIMENSIONAL METRICS IN R

## 2. How to get started

or

In [4]:
Pkg.rm("HDMjl")

    Updating `C:\Users\User\.julia\environments\v1.8\Project.toml`


  [8de29b41] - HDMjl v0.0.11 `https://github.com/d2cml-ai/HDMjl.jl#main`
    Updating `C:\Users\User\.julia\environments\v1.8\Manifest.toml`


  [8de29b41] - HDMjl v0.0.11 `https://github.com/d2cml-ai/HDMjl.jl#main`
┌ Info: We haven't cleaned this depot up for a bit, running Pkg.gc()...
└ @ Pkg C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.8\Pkg\src\Pkg.jl:719


      Active manifest files: 2 found


      Active artifact files: 99 found
      Active scratchspaces: 13 found


     Deleted 234 package installations (598.311 MiB)
     Deleted 4 repos (9.640 MiB)


In [1]:
import Pkg; Pkg.add(url = "https://github.com/d2cml-ai/HDMjl.jl")

    Updating git-repo `https://github.com/d2cml-ai/HDMjl.jl`

    Updating registry at `C:\Users\User\.julia\registries\General`


    Updating git-repo `https://github.com/JuliaRegistries/General.git`


   Resolving 

package versions...


  No Changes to `C:\Users\User\.julia\environments\v1.8\Project.toml`


  No Changes to `C:\Users\User\.julia\environments\v1.8\Manifest.toml`

In [1]:
using CodecXz, RData, DataFrames, StatsModels, Statistics, Distributions, PrettyTables

In [2]:
import Pkg

In [2]:
include("E:/causal_ml/hdm_paper/prueba/HDMjl.jl/src/HDMjl.jl")

Main.HDMjl

In [1]:
using HDMjl

## 3. Prediction using Approximate Sparsity

### 3.2. A Joint Significance Test for Lasso Regression.

In [3]:
using Random
Random.seed!(1234);
n = 100;
p = 100;
s = 3;
X = randn(n, p);
beta = vcat(fill(5, s), zeros(p - s));
Y = X * beta + randn(n);

In [4]:
lasso_reg = HDMjl.rlasso(X, Y, post = false);

In [5]:
post_lasso_reg = HDMjl.rlasso(X, Y, post = true) #now use post-lasso
post_lasso_reg["coefficients"]'

1×101 adjoint(::Vector{Float64}) with eltype Float64:
 -0.00682754  5.00958  4.93178  5.17705  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

## 4. Inference on Target Regression Coefficients

### 4.1. Intuition for the Orthogonality Principle in Linear Models via Partialling Out.

In [7]:
using CSV, GLM
using DataFrames

In [696]:
include("E:/causal_ml/hdm_paper/prueba/HDMjl.jl/src/HDMjl.jl")

Main.HDMjl

In [5]:
url = "https://raw.githubusercontent.com/d2cml-ai/HDMjl.jl/prueba/data/4_1.csv"
dta = DataFrame(CSV.read(download(url), DataFrame));
n, p = size(dta);
y = dta[:,"y"];
d = dta[:,"d"];
x = Matrix(dta[:,3:end]);

In [8]:
full_fit = lm(hcat(ones(length(y)), Matrix(dta[:,2:end])), y);
DataFrame(Estimate = coef(full_fit)[2], Std_Error = stderror(full_fit)[2])

,Estimate,Std_Error
,Float64,Float64
1,0.978075,0.0137122


In [9]:
rY_1 = lm(hcat(ones(length(y)), Matrix(dta[:,3:end])), y);
rY = y - predict(rY_1)
rD_1 = lm(hcat(ones(length(y)), Matrix(dta[:,3:end])), d);
rD = d - predict(rD_1);

In [10]:
partial_fit_ls = lm(hcat(ones(length(y)), rD), rY)
DataFrame(Estimate = coef(partial_fit_ls)[2], Std_Error = stderror(partial_fit_ls)[2])

,Estimate,Std_Error
,Float64,Float64
1,0.978075,0.0136862


In [11]:
rY_1 = HDMjl.rlasso(hcat(ones(length(y)), Matrix(dta[:,3:end])), y);
rY = rY_1["residuals"]
rD_1 = HDMjl.rlasso(hcat(ones(length(y)), Matrix(dta[:,3:end])), d);
rD = rD_1["residuals"]
partial_fit_postlasso = lm(hcat(ones(length(y)), rD), rY)
DataFrame(Estimate = coef(partial_fit_postlasso)[2], Std_Error = stderror(partial_fit_postlasso)[2])

,Estimate,Std_Error
,Float64,Float64
1,0.972739,0.0136868


In [13]:
Eff = HDMjl.rlassoEffect(x, y, d, method = "partialling out");
HDMjl.r_summary(Eff);

Estimates and significance testing of the effect of target variables
  Row   Estimate.   Std. Error   t value   Pr(>|t|) 

    1    0.972739    0.0136868   71.0715        0.0
---
Signif. codes:
0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [15]:
HDMjl.r_print(Eff)

Coefficients:

      1 

  0.973


In [16]:
Eff = HDMjl.rlassoEffect(x, y, d, method = "double selection");
HDMjl.r_summary(Eff);

Estimates and significance testing of the effect of target variables
  Row   Estimate.   Std. Error   t value   Pr(>|t|) 

    1    0.978075    0.0141562   69.0914        0.0
---
Signif. codes:
0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


### 4.2. Inference: Confidence Intervals and Significance Testing. The function rlassoEffects

In [1092]:
# using Random, Distributions, PrettyTables, DataFrames
# Random.seed!(1234);
# n = 100;
# p = 100;
# s = 3;
# x = randn(n, p);
# beta = vcat(fill(3, s), zeros(p - s));
# y =1 .+ x * beta + randn(n);

In [1170]:
include("E:/causal_ml/hdm_paper/prueba/HDMjl.jl/src/HDMjl.jl")

Main.HDMjl

In [357]:
using CSV
url = "https://raw.githubusercontent.com/d2cml-ai/HDMjl.jl/prueba/data/4_2.csv"
data = DataFrame(CSV.read(download(url), DataFrame));
n, p = size(data);
y = data[:,1];
#d = dta[:,"d"];
x = Matrix(data[:,2:end]);

In [358]:
lasso_effects = HDMjl.rlassoEffects(x, y, index = [1,2,3,50]);

In [360]:
HDMjl.r_print(lasso_effects, digits = 4)

Coefficients:

     X1          X2          X3          X50    

   2.9445      3.0413      2.9754       0.072


In [1180]:
HDMjl.r_summary(lasso_effects);

Estimates and significance testing of the effect of target variables
        Estimate.   Std. Error    t value       Pr(>|t|) 

   X1     2.94448    0.0881468    33.4043    1.1892e-244
   X2     3.04127     0.083891    36.2527    9.0141e-288
   X3      2.9754    0.0780394     38.127   4.58085e-318
  X50   0.0719553    0.0776455   0.926716       0.354074
---
Signif. codes:
0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [1179]:
HDMjl.r_confint(lasso_effects);

              2.5%      97.5% 

   X1      2.77171    3.11724
   X2      2.87685     3.2057
   X3      2.82245    3.12836
  X50   -0.0802271   0.224138


In [1186]:
HDMjl.r_confint(lasso_effects, joint = true);

             2.5%      97.5% 

   X1     2.72895    3.16001
   X2     2.83806    3.24448
   X3      2.7842     3.1666
  X50   -0.114585   0.258496


### 4.3. Application: the effect of gender on wage

In [18]:
include("E:/causal_ml/hdm_paper/prueba/HDMjl.jl/src/HDMjl.jl")

Main.HDMjl

In [127]:
url = "https://github.com/cran/hdm/raw/master/data/cps2012.rda"
cps2012 = load(download(url))["cps2012"];
n, p = size(cps2012);
size(cps2012)

(29217, 23)

In [380]:
x_formula = @formula(lnw ~ -1 + female + female*widowed + female*divorced + female*separated + female*nevermarried +
                    female*hsd08 + female*hsd911 + female*hsg + female*cg + female*ad + female*mw + female*so + female*we + female*exp1 + female*exp2 + female*exp3)
x_dframe = ModelFrame( x_formula, cps2012)
x1 = ModelMatrix(x_dframe)
X = x1.m[:,Not(1:16)];
X = hcat(x1.m[:,1:16], X)
size(X)
fom_1 =     ["widowed", "divorced", "separated", "nevermarried", "hsd08", "hsd911", "hsg", "cg", "ad", "mw", "so",
            "we", "exp1", "exp2", "exp3"];
data = cps2012[:,fom_1];
sub_data = ones(size(data)[1])
for i in 1:size(data)[2]
    if i <= (size(data)[2] -1)
        sub_data = hcat(sub_data, Matrix(data[:, i] .* data[:, Not(1:i)]) )
    end
end
sub_data = sub_data[:,2:end]
size(sub_data)
x = hcat(X, sub_data)
size(x)
filter = var.(eachcol(x)) .!= 0
x = x[:,filter]
print(size(x))
index_gender = [1,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31];
y = cps2012.lnw;
;

(29217, 116)

In [389]:
effects_female = HDMjl.rlassoEffects(x, y, index = index_gender);

In [390]:
HDMjl.r_summary(effects_female);

Estimates and significance testing of the effect of target variables
          Estimate.   Std. Error      t value      Pr(>|t|) 

   X1     -0.154923    0.0501624     -3.08843    0.00201216
  X17      0.136095    0.0906626      1.50112      0.133325
  X18      0.136939    0.0221817      6.17353    6.6782e-10
  X19     0.0233028    0.0532118     0.437925      0.661441
  X20      0.186853    0.0199424      9.36966   7.27651e-21
  X21     0.0278103     0.120914         0.23      0.818092
  X22     -0.119335    0.0518797     -2.30023     0.0214354
  X23    -0.0128898    0.0192232    -0.670533      0.502518
  X24     0.0101386    0.0183265     0.553218      0.580114
  X25    -0.0304637    0.0218061     -1.39703      0.162405
  X26   -0.00106344    0.0191918   -0.0554112      0.955811
  X27   -0.00818334    0.0193568    -0.422763      0.672468
  X28   -0.00422613    0.0211684    -0.199643       0.84176
  X29    0.00493526   0.00780428     0.632379      0.527139
  X30     -0.159519    0.0452

In [399]:
joint_CI = HDMjl.r_confint(effects_female, 0.95, joint = true);
joint_CI;

              2.5%        97.5% 

   X1    -0.292471   -0.0173751
  X17    -0.130276     0.402467
  X18     0.075579       0.1983
  X19    -0.114955      0.16156
  X20     0.129983     0.243724
  X21    -0.369439      0.42506
  X22    -0.267141     0.028471
  X23   -0.0644819    0.0387024
  X24   -0.0410296    0.0613067
  X25   -0.0950142    0.0340867
  X26   -0.0540328     0.051906
  X27   -0.0620344    0.0456677
  X28    -0.064856    0.0564037
  X29   -0.0162257    0.0260962
  X30    -0.282576   -0.0364624
  X31    0.0171196    0.0597815


### 4.4. Application: Estimation of the treatment effect in a linear model with many confounding factors

In [12]:
using LinearAlgebra

In [14]:
include("E:/causal_ml/hdm_paper/prueba/HDMjl.jl/src/HDMjl.jl")

Main.HDMjl

In [4]:
using GLM
using CodecXz
using RData
using DataFrames
url = "https://github.com/cran/hdm/raw/master/data/GrowthData.rda";
GrowthData = load(download(url))["GrowthData"];
y = GrowthData[:, 1];
d = GrowthData[:, 3];
X = Matrix(GrowthData[:, Not(1, 2, 3)]);
X_1 = Matrix(GrowthData[:, Not(1, 2)]);

In [13]:
Q, R = qr(hcat(ones(length(y)), X_1))
β = pinv(hcat(ones(length(y)), X_1)) * y
# ls_effect = β;
res = y - hcat(ones(length(y)), X_1) * β;
n = size(hcat(ones(length(y)), X_1))[1]
k = size(hcat(ones(length(y)), X_1))[2]

sigma2_hat = (res' * res) / (n - k)
vcov_beta_hat = sigma2_hat .* inv(hcat(ones(length(y)), X_1)' * hcat(ones(length(y)), X_1));
se = sqrt.(diag(vcov_beta_hat))

ls_effect = DataFrame(Estimate = β, stderror = se);

In [180]:
# ls_effect = lm(hcat(ones(length(y)), X_1)[:, 1:42], y);
# coef(ls_effect)

In [6]:
lasso_effect = HDMjl.rlassoEffect(X, y, d, method = "partialling out");
HDMjl.r_summary(lasso_effect);

Estimates and significance testing of the effect of target variables


  Row    Estimate.   Std. Error    t value      Pr(>|t|) 

    1   -0.0498115    0.0139364   -3.57421   0.000351288
---


Signif. codes:
0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [7]:
doublesel_effect = HDMjl.rlassoEffect(X, y, d, method = "double selection");
HDMjl.r_summary(doublesel_effect);

Estimates and significance testing of the effect of target variables
  Row    Estimate.   Std. Error    t value     Pr(>|t|) 

    1   -0.0500059    0.0157914   -3.16666   0.00154203
---
Signif. codes:
0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [25]:
using PrettyTables
table = zeros(3,2)
table[1,:] = [round(Matrix(ls_effect)[2,1], digits = 2), round.(Matrix(ls_effect)[2,2], digits = 5)]
table[2,:] = [round(lasso_effect.coefficients, digits =2), round(lasso_effect.se, digits = 5)]
table[3,:] = [round(doublesel_effect.coefficients, digits =2), round(doublesel_effect.se, digits = 5)];

In [26]:
# table1 = DataFrame(table, :auto)
# table1 = rename(table1, ["Estimate", "Std. Error"])
index = ["full reg via ols", "partial reg
via post-lasso ", "partial reg via double selection"]
pretty_table(hcat(index, table), show_row_number = false, header = [" ", "Estimate", "Std. Error"], tf = tf_simple, nosubheader = true)

=================================== ========== =============
                                     Estimate   Std. Error 
=================================== ========== =============
                  full reg via ols      -0.01      0.02989
      partial reg\nvia post-lasso       -0.05      0.01394
  partial reg via double selection      -0.05      0.01579
=================================== ========== =============


## 5. Instrumental Variable Estimation in a High-Dimensional Setting

### 5.2. Application: Economic Development and Institutions.

In [27]:
using RDatasets, TexTables, LinearAlgebra, GLM

In [28]:
include("E:/causal_ml/hdm_paper/prueba/HDMjl.jl/src/HDMjl.jl")

Main.HDMjl

In [29]:
using Statistics, StatsModels, RData
url = "https://github.com/cran/hdm/raw/master/data/AJR.rda";
AJR = load(download(url))["AJR"];
y = AJR[!,"GDP"]
d = AJR[!,"Exprop"]
z = AJR[!,"logMort"];
x_formula = @formula(GDP ~ -1 + Latitude + Latitude2 + Africa + Asia + Namer + Samer
    + Latitude*Latitude2 + Latitude*Africa + Latitude*Asia + Latitude*Namer + Latitude*Samer
    + Latitude2*Africa + Latitude2*Asia + Latitude2*Namer + Latitude2*Samer
    + Africa*Asia + Africa*Namer + Africa*Samer
    + Asia*Namer + Asia*Samer
    + Namer*Samer)
x_dframe = ModelFrame( x_formula, AJR)
x1 = ModelMatrix(x_dframe)
x = x1.m
size(x)

(64, 21)

In [31]:
AJR_Xselect  = HDMjl.rlassoIV(x, d, y, z, select_X=true, select_Z=false);

In [33]:
HDMjl.r_print(AJR_Xselect)

Coefficients:

      d1     

    0.845


In [35]:
HDMjl.r_summary(AJR_Xselect);

Estimates and Significance Testing of the effect of target variables in the IV regression model
         coeff.        se.   t-value      p-value 

  d1   0.845027   0.269926   3.13059   0.00174458
---
Signif. codes:
0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [36]:
HDMjl.r_confint(AJR_Xselect);

           2.5%     97.5% 

  d1   0.315981   1.37407


In [21]:
rY_1 = lm(hcat(ones(length(y)), Matrix(AJR[:,3:end])), y);
rY = y - predict(rY_1)
rD_1 = lm(hcat(ones(length(y)), Matrix(AJR[:,3:end])), d);
rD = d - predict(rD_1);
rZ_1 = lm(hcat(ones(length(y)), Matrix(AJR[:,3:end])), z);
rZ = z - predict(rZ_1);

In [22]:
rY_1 = lm(@formula(GDP ~ Latitude + Latitude2 + Africa + Asia + Namer + Samer + Latitude*Latitude2 + Latitude*Africa + Latitude*Asia + Latitude*Namer + Latitude*Samer
          + Latitude2*Africa + Latitude2*Asia + Latitude2*Namer + Latitude2*Samer + Africa*Asia + Africa*Namer + Africa*Samer
          + Asia*Namer + Asia*Samer + Namer*Samer), AJR)
rY = y - predict(rY_1)

rD_1 = lm(@formula(Exprop ~ Latitude + Latitude2 + Africa + Asia + Namer + Samer + Latitude*Latitude2 + Latitude*Africa + Latitude*Asia + Latitude*Namer + Latitude*Samer
          + Latitude2*Africa + Latitude2*Asia + Latitude2*Namer + Latitude2*Samer + Africa*Asia + Africa*Namer + Africa*Samer
          + Asia*Namer + Asia*Samer + Namer*Samer), AJR)
rD = d - predict(rD_1)

rZ_1 = lm(@formula(logMort ~ Latitude + Latitude2 + Africa + Asia + Namer + Samer + Latitude*Latitude2 + Latitude*Africa + Latitude*Asia + Latitude*Namer + Latitude*Samer
          + Latitude2*Africa + Latitude2*Asia + Latitude2*Namer + Latitude2*Samer + Africa*Asia + Africa*Namer + Africa*Samer
          + Asia*Namer + Asia*Samer + Namer*Samer), AJR)
rZ = z - predict(rZ_1);

In [23]:
ivfit_lm = HDMjl.tsls(rD, rY, rZ, nothing, intercept=false)
DataFrame(Estimate = ivfit_lm["coefficients"][1,2], Std_Error = ivfit_lm["se"])

,Estimate,Std_Error
,Float64,Float64
1,1.26721,1.73054


In [24]:
x_formula1 = @formula(GDP ~ Latitude + Latitude2 + Africa + Asia + Namer + Samer
    + Latitude*Latitude2 + Latitude*Africa + Latitude*Asia + Latitude*Namer + Latitude*Samer
    + Latitude2*Africa + Latitude2*Asia + Latitude2*Namer + Latitude2*Samer
    + Africa*Asia + Africa*Namer + Africa*Samer
    + Asia*Namer + Asia*Samer
    + Namer*Samer)
x_dframe1 = ModelFrame( x_formula, AJR)
x1_1 = ModelMatrix(x_dframe)
xx = x1.m;

In [35]:
rY_1 = HDMjl.rlasso(xx, y);
rY = rY_1["residuals"]
rD_1 = HDMjl.rlasso(xx, d);
rD = rD_1["residuals"]
rZ_1 = HDMjl.rlasso(xx, z);
rZ = rZ_1["residuals"]

ivfit_lasso = HDMjl.tsls(rD, rY, rZ)
DataFrame(Estimate = ivfit_lasso["coefficients"][1,2], Std_Error = ivfit_lasso["se"][1])

,Estimate,Std_Error
,Float64,Float64
1,0.845027,0.272094


### 5.3. Application: Impact of Eminent Domain Decisions on Economic Outcomes.

In [67]:
using Crayons, Distributions, GLM

In [136]:
include("E:/causal_ml/hdm_paper/prueba/HDMjl.jl/src/HDMjl.jl")

Main.HDMjl

In [137]:
using Statistics, GLM
url = "https://github.com/cran/hdm/raw/master/data/EminentDomain.rda";
EminentDomain = load(download(url))["EminentDomain"];
z = EminentDomain["logGDP"]["z"];
x = EminentDomain["logGDP"]["x"];
d = EminentDomain["logGDP"]["d"];
y = EminentDomain["logGDP"]["y"];
x = x[:, (mean(x, dims = 1) .> 0.05)'];
z = z[:, (mean(z, dims = 1) .> 0.05)'];

In [138]:
ED_ols = lm(hcat(ones(length(vec(y))), hcat(d, x)), vec(y));
ED_2sls = HDMjl.tsls(d, y, z[:,1:2], x, intercept = false);

In [141]:
lasso_IV_Z = HDMjl.rlassoIV(x, d, y, z, select_X = false, select_Z = true);

In [140]:
HDMjl.r_summary(lasso_IV_Z);

Estimates and Significance Testing of the effect of target variables in the IV regression model
         coeff.        se.   t-value    p-value 

  d1   0.414602   0.290249   1.42843   0.153167
---
Signif. codes:
0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [142]:
HDMjl.r_confint(lasso_IV_Z);

            2.5%     97.5% 

  d1   -0.154276   0.98348


In [143]:
HDMjl.r_print(lasso_IV_Z)

Coefficients:

      d1     

    0.415


In [144]:
lasso_IV_XZ = HDMjl.rlassoIV(x, d, y, z, select_X = true, select_Z = true);

In [145]:
HDMjl.r_summary(lasso_IV_XZ);

Estimates and Significance Testing of the effect of target variables in the IV regression model
           coeff.        se.     t-value    p-value 

  d1   -0.0238347   0.128507   -0.185475   0.852857
---
Signif. codes:
0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [146]:
HDMjl.r_confint(lasso_IV_XZ);

            2.5%      97.5% 

  d1   -0.275703   0.228033


In [147]:
HDMjl.r_print(lasso_IV_XZ)

Coefficients:

      d1     

    -0.024


In [152]:
Matrix(HDMjl.r_summary(lasso_IV_XZ)[:, 2:3])

Estimates and Significance Testing of the effect of target variables in the IV regression model
           coeff.        se.     t-value    p-value 

  d1   -0.0238347   0.128507   -0.185475   0.852857
---
Signif. codes:
0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


1×2 Matrix{Any}:
 -0.0238347  0.128507

In [155]:
using PrettyTables
table = zeros(4,2)
table[1,:] = [coef(ED_ols)[2], stderror(ED_ols)[2]]
table[2,:] = [ED_2sls["coefficients"][1,2], ED_2sls["se"][1]]
table[3,:] = Matrix(HDMjl.r_summary(lasso_IV_Z)[:,2:3]);
table[4, :] = Matrix(HDMjl.r_summary(lasso_IV_XZ)[:, 2:3]);

Estimates and Significance Testing of the effect of target variables in the IV regression model
         coeff.        se.   t-value    p-value 

  d1   0.414602   0.290249   1.42843   0.153167
---
Signif. codes:
0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
Estimates and Significance Testing of the effect of target variables in the IV regression model
           coeff.        se.     t-value    p-value 

  d1   -0.0238347   0.128507   -0.185475   0.852857
---
Signif. codes:
0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [171]:
# table1 = DataFrame(table, :auto)
# table1 = rename(table1, ["Estimate", "Std. Error"])
index = ["ols regression", "IV estimation ", "selection on Z", "selection on X and Z"]
pretty_table(hcat(index, table), show_row_number = false, header = [" ", "Estimate", "Std. Error"], tf = tf_simple, nosubheader = true)

======================= ============ =============
                           Estimate   Std. Error 
======================= ============ =============
        ols regression   0.00786473   0.00986593
        IV estimation    -0.0107097    0.0337652
        selection on Z     0.414602     0.290249
  selection on X and Z   -0.0238347     0.128507
======================= ============ =============


## 6. Inference on Treatment Effects in a High-Dimensional Setting

### 6.3. Application: 401(k) plan participation.

In [26]:
url = "https://github.com/cran/hdm/raw/master/data/pension.rda";
pension = load(download(url))["pension"];
y = pension[:, "tw"];
d = pension[:, "p401"];
z = pension[:, "e401"];
X = Matrix(pension[:, ["i2", "i3", "i4", "i5", "i6", "i7", "a2", "a3", "a4", "a5", "fsize", "hs", "smcol", "col", "marr", "twoearn", "db", "pira", "hown"]]);
rlassoATE(X, d, y)

Dict{String, Any} with 5 entries:
  "se"          => 1930.68
  "individual"  => [-30618.3, -57537.6, -71442.9, 21383.3, -2.32925e5, 3.40765e…
  "sample_size" => 9915
  "te"          => 10180.1
  "type"        => "ATE"

In [51]:
rlassoATET(X, d, y)

Dict{String, Any} with 5 entries:
  "se"          => 2944.43
  "individual"  => [-21536.4, -52877.2, -1.44867e5, -2739.29, -307741.0, 7.3912…
  "sample_size" => 9915
  "te"          => 12628.5
  "type"        => "ATET"

In [52]:
rlassoLATE(X, d, y, z)

Dict{String, Any} with 5 entries:
  "se"          => 2326.9
  "individual"  => [-50526.8, -1.39158e5, -1.37102e5, 38508.0, -6.5644e5, 7.943…
  "sample_size" => 9915
  "te"          => 12992.1
  "type"        => "LATE"

In [53]:
rlassoLATET(X, d, y, z)

Dict{String, Any} with 5 entries:
  "se"          => 3645.28
  "individual"  => [-35580.5, -90558.0, -1.83628e5, -5303.13, -8.0766e5, 1.8866…
  "sample_size" => 9915
  "te"          => 15323.2
  "type"        => "LATET"

## 7. The Lasso Methods for Discovery of Significant Causes amongst Many Potential Causes, with Many Controls


In [686]:
include("E:/causal_ml/hdm_paper/prueba/HDMjl.jl/src/HDMjl.jl")

Main.HDMjl

In [173]:
using CSV
url = "https://raw.githubusercontent.com/d2cml-ai/HDMjl.jl/prueba/data/7_.csv"
data = DataFrame(CSV.read(download(url), DataFrame));
n, p = size(data);
p1 = 20;
X = data[:,2:end]
Y = data[:,1];

In [178]:
HDMjl.r_confint(HDMjl.rlassoEffects(X, Y, index = [1:p1;]), joint = true);

              2.5%      97.5% 

   X1      4.50675    5.22214
   X2    -0.321224   0.311879
   X3    -0.358448   0.192826
   X4    -0.260307   0.293453
   X5     -0.28277   0.282461
   X6    -0.328361    0.30112
   X7    -0.232153   0.306844
   X8   -0.0531874   0.479497
   X9    -0.193021   0.396693
  X10    -0.242849   0.269725
  X11    -0.320578   0.215327
  X12    -0.315627   0.272158
  X13    -0.180324   0.382993
  X14    -0.331511    0.39337
  X15    -0.329087   0.320235
  X16    -0.271623   0.337679
  X17    -0.185883   0.423636
  X18    -0.373985    0.05162
  X19     -0.11292   0.399297
  X20    -0.220993   0.260713
